In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("Basics").getOrCreate()

23/03/28 11:25:41 WARN Utils: Your hostname, TIGER03531 resolves to a loopback address: 127.0.1.1; using 172.29.67.58 instead (on interface eth0)
23/03/28 11:25:41 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/28 11:25:42 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df = spark.read.json("2015-summary.json")

In [4]:
df.show()

+--------------------+-------------------+-----+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+--------------------+-------------------+-----+
|       United States|            Romania|   15|
|       United States|            Croatia|    1|
|       United States|            Ireland|  344|
|               Egypt|      United States|   15|
|       United States|              India|   62|
|       United States|          Singapore|    1|
|       United States|            Grenada|   62|
|          Costa Rica|      United States|  588|
|             Senegal|      United States|   40|
|             Moldova|      United States|    1|
|       United States|       Sint Maarten|  325|
|       United States|   Marshall Islands|   39|
|              Guyana|      United States|   64|
|               Malta|      United States|    1|
|            Anguilla|      United States|   41|
|             Bolivia|      United States|   30|
|       United States|           Paraguay|    6|
|             Algeri

## Schema

In [5]:
df.printSchema()

root
 |-- DEST_COUNTRY_NAME: string (nullable = true)
 |-- ORIGIN_COUNTRY_NAME: string (nullable = true)
 |-- count: long (nullable = true)



In [6]:
from pyspark.sql.types import StructField, StructType, StringType, LongType

In [7]:
myManualSchema = StructType([
StructField("DEST_COUNTRY_NAME", StringType(), True),
StructField("ORIGIN_COUNTRY_NAME", StringType(), True),
StructField("count", LongType(), False, metadata={"hello":"world"})
])


In [8]:
df2 = df = spark.read.json("2015-summary.json", schema= myManualSchema)


In [9]:
df2.show(5)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|   15|
|    United States|            Croatia|    1|
|    United States|            Ireland|  344|
|            Egypt|      United States|   15|
|    United States|              India|   62|
+-----------------+-------------------+-----+
only showing top 5 rows



## Columns and Expression

In [10]:
from pyspark.sql.functions import col, column

In [11]:
col("someColumnName")

Column<'someColumnName'>

In [12]:
column("someColumnName")

Column<'someColumnName'>

In [13]:
df.select("count").show(5)

+-----+
|count|
+-----+
|   15|
|    1|
|  344|
|   15|
|   62|
+-----+
only showing top 5 rows



In [14]:
df.columns

['DEST_COUNTRY_NAME', 'ORIGIN_COUNTRY_NAME', 'count']

In [15]:
from pyspark.sql.functions import expr
df.withColumn('timepass', expr("(count + 2) > 10")).show()

+--------------------+-------------------+-----+--------+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|timepass|
+--------------------+-------------------+-----+--------+
|       United States|            Romania|   15|    true|
|       United States|            Croatia|    1|   false|
|       United States|            Ireland|  344|    true|
|               Egypt|      United States|   15|    true|
|       United States|              India|   62|    true|
|       United States|          Singapore|    1|   false|
|       United States|            Grenada|   62|    true|
|          Costa Rica|      United States|  588|    true|
|             Senegal|      United States|   40|    true|
|             Moldova|      United States|    1|   false|
|       United States|       Sint Maarten|  325|    true|
|       United States|   Marshall Islands|   39|    true|
|              Guyana|      United States|   64|    true|
|               Malta|      United States|    1|   false|
|            A

## Records and Rows

In [16]:
df.first()

Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Romania', count=15)

In [17]:
df.collect()[200]

Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Switzerland', count=305)

In [18]:
rows = df.count()
columns = len(df.columns)
print(rows, columns)

256 3


## Create Row

In [19]:
from pyspark.sql import Row
new_row = Row("United States","India", 5000)

In [20]:
new_row = spark.createDataFrame([new_row], df.schema)

## Dataframe Transformation

### Appending row to dataframe

In [21]:
df = df.union(new_row)

### Creating seperate index column

In [22]:
from pyspark.sql.functions import row_number
from pyspark.sql.window import Window
from pyspark.sql.functions import monotonically_increasing_id
df = df.withColumn("Index", row_number().over(Window.orderBy(monotonically_increasing_id())) - 1)

In [23]:
df.orderBy('Index', ascending=False).show(10)

23/03/28 11:25:54 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/03/28 11:25:54 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/03/28 11:25:54 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


23/03/28 11:25:56 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/03/28 11:25:56 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
+--------------------+--------------------+-----+-----+
|   DEST_COUNTRY_NAME| ORIGIN_COUNTRY_NAME|count|Index|
+--------------------+--------------------+-----+-----+
|       United States|               India| 5000|  256|
|              Greece|       United States|   30|  255|
|Bonaire, Sint Eus...|       United States|   58|  254|
|       United States|               Haiti|  225|  253|
|             Uruguay|       United States|   43|  252|
|       United States|Saint Kitts and N...|  145|  251|
|            Bulgaria|       United States|    3|  250|
|        Cook Islands|       United States|   13|  249|
|       United States|             Bolivia|   13|  248

In [24]:
df.createOrReplaceTempView('plane_table')

In [25]:
spark.sql("select * from plane_table").show()

23/03/28 11:25:57 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/03/28 11:25:57 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/03/28 11:25:57 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
+--------------------+-------------------+-----+-----+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|Index|
+--------------------+-------------------+-----+-----+
|       United States|            Romania|   15|    0|
|       United States|            Croatia|    1|    1|
|       United States|            Ireland|  344|    2|
|               Egypt|      United States|   15|    3|
|       United States|              India|   62|    4|
|       United States|          Singapore|    1|    5|
|       Un

## Select and select expr

In [26]:
df.select("DEST_COUNTRY_NAME").show(5)

+-----------------+
|DEST_COUNTRY_NAME|
+-----------------+
|    United States|
|    United States|
|    United States|
|            Egypt|
|    United States|
+-----------------+
only showing top 5 rows



In [27]:
spark.sql("""select * from plane_table
where count > 10
limit 15;""").show()

23/03/28 11:25:58 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/03/28 11:25:58 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/03/28 11:25:58 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/03/28 11:25:58 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
+--------------------+--------------------+-----+-----+
|   DEST_COUNTRY_NAME| ORIGIN_COUNTRY_NAME|count|Index|
+--------------------+--------------------+-----+-----+
|       United States|             Romania|   15|    0|
|       United States|             Ireland|  344|    2|
|               Egypt|       United States|   15|    3|
|   

In [28]:
spark.sql("""select * from plane_table
where count > 10
limit 15;""").count()

15

## Adding Columns 

In [29]:
from pyspark.sql.functions import mean, stddev

In [30]:
mean = df.select(mean(df['count'])).collect()[0][0]
mean

1783.330739299611

In [31]:
std = df.select(stddev(df['count'])).collect()[0][0]
std

23082.182700056277

In [32]:
df = df.withColumn("Standardized count", (df['count'] - mean) / std)

In [33]:
df.show()

23/03/28 11:26:00 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/03/28 11:26:00 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/03/28 11:26:00 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
+--------------------+-------------------+-----+-----+--------------------+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|Index|  Standardized count|
+--------------------+-------------------+-----+-----+--------------------+
|       United States|            Romania|   15|    0|-0.07661020460146083|
|       United States|            Croatia|    1|    1|-0.07721673302998618|
|       United States|            Ireland|  344|    2|-0.06235678653111526|
|               Egypt|      United States|   15| 

In [34]:
from pyspark.sql.functions import sum
df.select(sum(df['Standardized count']) / df.count()).show()

+-------------------------------+
|(sum(Standardized count) / 257)|
+-------------------------------+
|           1.220381340298080...|
+-------------------------------+



In [35]:
df.select(stddev(df['Standardized count'])).collect()[0][0]

1.0000000000000002

## Renaming Columns

In [36]:
df = df.withColumnRenamed('count','Count')

In [37]:
df.show(5)

23/03/28 11:26:02 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/03/28 11:26:02 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/03/28 11:26:02 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
+-----------------+-------------------+-----+-----+--------------------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|Count|Index|  Standardized count|
+-----------------+-------------------+-----+-----+--------------------+
|    United States|            Romania|   15|    0|-0.07661020460146083|
|    United States|            Croatia|    1|    1|-0.07721673302998618|
|    United States|            Ireland|  344|    2|-0.06235678653111526|
|            Egypt|      United States|   15|    3|-0.0766102046014

## Changing Column Type

In [38]:
df = df.withColumn("count2", col("count").cast("float"))
df.show(5)

23/03/28 11:26:03 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/03/28 11:26:03 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/03/28 11:26:03 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
+-----------------+-------------------+-----+-----+--------------------+------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|Count|Index|  Standardized count|count2|
+-----------------+-------------------+-----+-----+--------------------+------+
|    United States|            Romania|   15|    0|-0.07661020460146083|  15.0|
|    United States|            Croatia|    1|    1|-0.07721673302998618|   1.0|
|    United States|            Ireland|  344|    2|-0.06235678653111526| 344.0|
|            Egypt|      

## Removing a Column

In [39]:
df = df.drop('count2')
df.show()

23/03/28 11:26:03 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/03/28 11:26:03 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/03/28 11:26:03 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
+--------------------+-------------------+-----+-----+--------------------+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|Count|Index|  Standardized count|
+--------------------+-------------------+-----+-----+--------------------+
|       United States|            Romania|   15|    0|-0.07661020460146083|
|       United States|            Croatia|    1|    1|-0.07721673302998618|
|       United States|            Ireland|  344|    2|-0.06235678653111526|
|               Egypt|      United States|   15| 

## Filtering Rows

In [40]:
df.filter((df['count'] > 10) & (df['ORIGIN_COUNTRY_NAME'] == 'India')).show()

23/03/28 11:26:04 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/03/28 11:26:04 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/03/28 11:26:04 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/03/28 11:26:04 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/03/28 11:26:04 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
+-----------------+-------------------+-----+-----+--------------------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|Count|Index|  Standardized count|
+-----------------+----------

## Getting Unique Rows

In [41]:
df.select("ORIGIN_COUNTRY_NAME").distinct().show()

+--------------------+
| ORIGIN_COUNTRY_NAME|
+--------------------+
|            Paraguay|
|              Russia|
|            Anguilla|
|             Senegal|
|              Sweden|
|            Kiribati|
|              Guyana|
|         Philippines|
|           Singapore|
|            Malaysia|
|                Fiji|
|              Turkey|
|             Germany|
|              Jordan|
|               Palau|
|Turks and Caicos ...|
|              France|
|              Greece|
|British Virgin Is...|
|              Taiwan|
+--------------------+
only showing top 20 rows



In [42]:
df.select("ORIGIN_COUNTRY_NAME").distinct().count()

125

In [43]:
df.select("ORIGIN_COUNTRY_NAME", "DEST_COUNTRY_NAME").distinct().count()

256

## Sorting

In [44]:
df.orderBy(col("DEST_COUNTRY_NAME").desc(), col("count").desc()).show()

23/03/28 11:26:06 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/03/28 11:26:06 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/03/28 11:26:06 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/03/28 11:26:06 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/03/28 11:26:06 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
+-----------------+-------------------+------+-----+--------------------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME| Count|Index|  Standardized count|
+-----------------+--------

## Limit

In [45]:
df.limit(5).show()

23/03/28 11:26:06 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/03/28 11:26:06 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/03/28 11:26:06 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
+-----------------+-------------------+-----+-----+--------------------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|Count|Index|  Standardized count|
+-----------------+-------------------+-----+-----+--------------------+
|    United States|            Romania|   15|    0|-0.07661020460146083|
|    United States|            Croatia|    1|    1|-0.07721673302998618|
|    United States|            Ireland|  344|    2|-0.06235678653111526|
|            Egypt|      United States|   15|    3|-0.0766102046014

## Repartition and Coalesce

In [46]:
df.rdd.getNumPartitions()

23/03/28 11:26:07 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/03/28 11:26:07 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/03/28 11:26:07 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/03/28 11:26:07 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/03/28 11:26:07 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


1

In [47]:
df.repartition(5).show()

23/03/28 11:26:07 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/03/28 11:26:07 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/03/28 11:26:07 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/03/28 11:26:07 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
+-----------------+-------------------+-----+-----+--------------------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|Count|Index|  Standardized count|
+-----------------+-------------------+-----+-----+--------------------+
|    United States|              Haiti|  225|  253|-0.06751227817358069|
|            Qatar|      United States|  108|  1

In [48]:
df.repartition(col("DEST_COUNTRY_NAME")).show()

23/03/28 11:26:07 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/03/28 11:26:07 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/03/28 11:26:08 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/03/28 11:26:08 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
+--------------------+-------------------+-----+-----+--------------------+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|Count|Index|  Standardized count|
+--------------------+-------------------+-----+-----+--------------------+
|            Anguilla|      United States|   41|   14| -0.0754837946627709|
|              Russia|      United S

In [49]:
df.repartition(5, col("DEST_COUNTRY_NAME")).coalesce(2).show()

23/03/28 11:26:08 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/03/28 11:26:08 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/03/28 11:26:08 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/03/28 11:26:08 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
+--------------------+-------------------+-----+-----+--------------------+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|Count|Index|  Standardized count|
+--------------------+-------------------+-----+-----+--------------------+
|             Moldova|      United States|    1|    9|-0.07721673302998618|
|             Bolivia|      United S

In [73]:
def filter(df, filter_expr):
    '''
    This function will filter the snowpark dataframe by passing in the filter expression
    
    Parameters
    ----------
    df           : Snowpark Dataframe
                 : Dataframe type must be snowflake.snowpark.DataFrame
                 
    filter_expr  : Expression for Filtering the Dataframe
                 : Format like (df['col1'] == 1) & (df['col2'] >= 0) | (df['col3'] == 'Name')
                   The conditions must always be enclosed by brackets '()' or else the function will not work.
    '''
    result = df.filter(filter_expr)
    return result

In [74]:
filter_expression = (df['DEST_COUNTRY_NAME'] == 'India') | (df['Count'] >= 200)

In [71]:
df2 = filter(df, filter_expression)
df2.sort(df2['Count']).show()

23/03/28 11:46:54 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/03/28 11:46:54 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/03/28 11:46:54 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/03/28 11:46:54 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/03/28 11:46:54 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
+--------------------+--------------------+-----+-----+--------------------+
|   DEST_COUNTRY_NAME| ORIGIN_COUNTRY_NAME|Count|Index|  Standardized count|
+--------------------

In [75]:
type(df)

pyspark.sql.dataframe.DataFrame

In [86]:
def freq_table(df, col_name):
    from pyspark.sql.functions import col, sum, round
    from pyspark.sql.window import Window
    
    freq_df = df.groupBy(col_name).count().select(col(col_name), col('count').alias('Count'))
    total = df.count()
    freq_df = freq_df.withColumn('Frequency', round(col('count'), 2))
    freq_df = freq_df.withColumn('% of the Total', round(col('count')/total*100, 2))
    freq_df = freq_df.withColumn('Cumulative Frequency', round(sum('count').over(Window.orderBy(col(col_name))).cast('float'), 2))
    freq_df = freq_df.withColumn('Cumulative %', round(sum('count').over(Window.orderBy(col(col_name))).cast('float')/total*100, 2))
    return freq_df

In [87]:
result = freq_table(df, "DEST_COUNTRY_NAME")
result.show()

23/03/28 12:25:22 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/03/28 12:25:22 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/03/28 12:25:22 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/03/28 12:25:22 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/03/28 12:25:22 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/03/28 12:25:22 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/03/28 1

In [ ]:
df.select(count(i))

In [88]:
from pyspark.sql.functions import count, isnan, sum, when
from pyspark.sql.window import Window
from pyspark.sql.types import DoubleType

def freq_with_cumulative_percentage(df, column_name):
    count_col_name = f"{column_name}_count"
    percent_col_name = f"{column_name}_percent"
    cumulative_freq_col_name = f"{column_name}_cumulative_freq"
    cumulative_percent_col_name = f"{column_name}_cumulative_percent"
    
    # create a DataFrame with counts and percentages
    freq_df = df.groupBy(column_name) \
                .agg(count("*").alias(count_col_name)) \
                .withColumn(percent_col_name, count("*") / df.count() * 100.0)
    
    # add a row for missing values
    freq_df = freq_df.union(df.select(when(isnan(column_name) | col(column_name).isNull(), "__missing__").alias(column_name)) \
                             .groupBy(column_name).agg(count("*").alias(count_col_name)) \
                             .withColumn(percent_col_name, sum(col(count_col_name)) / df.count() * 100.0))
    
    # create a window to compute the cumulative frequency and percentage
    window_spec = Window.orderBy(col(count_col_name).desc())
    cumulative_freq = sum(col(count_col_name)).over(window_spec)
    cumulative_percent = (cumulative_freq / df.count() * 100.0).cast(DoubleType())
    
    # add the cumulative frequency and percentage columns
    freq_df = freq_df.withColumn(cumulative_freq_col_name, cumulative_freq) \
                     .withColumn(cumulative_percent_col_name, cumulative_percent)
    
    # rename the columns
    freq_df = freq_df.select(column_name,
                             count_col_name.alias("Count"),
                             percent_col_name.alias("Percent"),
                             cumulative_freq_col_name.alias("Cumulative_Frequency"),
                             cumulative_percent_col_name.alias("Cumulative_Percent"))
    
    return freq_df


In [89]:
final = freq_with_cumulative_percentage(df, "DEST_COUNTRY_NAME")

AnalysisException: grouping expressions sequence is empty, and 'DEST_COUNTRY_NAME' is not an aggregate function. Wrap '(((CAST(count(1) AS DOUBLE) / CAST(257 AS DOUBLE)) * 100.0D) AS DEST_COUNTRY_NAME_percent)' in windowing function(s) or wrap 'DEST_COUNTRY_NAME' in first() (or first_value) if you don't care which value you get.;
Aggregate [DEST_COUNTRY_NAME#31, DEST_COUNTRY_NAME_count#1382L, ((cast(count(1) as double) / cast(257 as double)) * 100.0) AS DEST_COUNTRY_NAME_percent#1396]
+- Aggregate [DEST_COUNTRY_NAME#31], [DEST_COUNTRY_NAME#31, count(1) AS DEST_COUNTRY_NAME_count#1382L]
   +- Project [DEST_COUNTRY_NAME#31, ORIGIN_COUNTRY_NAME#32, Count#425L, Index#110, Standardized count#302]
      +- Project [DEST_COUNTRY_NAME#31, ORIGIN_COUNTRY_NAME#32, Count#425L, Index#110, Standardized count#302, cast(count#425L as float) AS count2#455]
         +- Project [DEST_COUNTRY_NAME#31, ORIGIN_COUNTRY_NAME#32, count#33L AS Count#425L, Index#110, Standardized count#302]
            +- Proje